In [3]:
import cv2
import csv
import pandas as pd 
import re
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageStat
from sklearn.model_selection import train_test_split
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image as imge
from keras.applications.resnet50 import preprocess_input, decode_predictions
import math
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth = True

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [5]:
ext = ".jpg"
width = 224
height = 224

FilePath = "/home/nramvinojen/Programs/Workbench/24Jan2019/"
#RunFolder = "Default"
col_names = ['label','feature1','feature2','feature3','feature4','feature5','feature6','feature7','feature8','feature9','feature10','feature11','feature12','feature13','feature14','feature15','feature16','feature17','feature18','feature19','feature20','feature21','feature22','feature23','feature24','feature25','feature26','feature27','feature28','feature29','feature30','feature31','feature32','feature33','feature34','feature35','feature36','feature37','feature38','feature39','feature40','feature41','feature42','feature43','feature44','feature45','feature46','feature47','feature48','feature49','feature50','feature51','feature52','feature53','feature54','feature55','feature56','feature57','feature58','feature59','feature60','feature61','feature62','feature63','feature64','feature65','feature66','feature67','feature68','feature69','feature70','feature71','feature72','feature73','feature74','feature75','feature76','feature77','feature78','feature79','feature80','feature81','feature82','feature83','feature84','feature85','feature86','feature87','feature88','feature89','feature90','feature91','feature92','feature93','feature94','feature95','feature96','feature97','feature98','feature99','feature100','feature101','feature102','feature103','feature104','feature105','feature106','feature107','feature108','feature109','feature110','feature111','feature112','feature113','feature114','feature115','feature116','feature117','feature118','feature119','feature120','feature121','feature122','feature123','feature124','feature125','feature126','feature127','feature128','feature129','feature130','feature131','feature132','feature133','feature134','feature135','feature136','feature137','feature138','feature139','feature140','feature141','feature142','feature143','feature144','feature145','feature146','feature147','feature148','feature149','feature150','feature151','feature152','feature153','feature154','feature155','feature156','feature157','feature158','feature159','feature160','feature161','feature162','feature163','feature164','feature165','feature166','feature167','feature168','feature169','feature170','feature171','feature172','feature173','feature174','feature175','feature176','feature177','feature178','feature179','feature180','feature181','feature182','feature183','feature184','feature185','feature186','feature187','feature188','feature189','feature190','feature191','feature192','feature193','feature194','feature195','feature196','feature197','feature198','feature199','feature200','feature201','feature202','feature203','feature204','feature205','feature206','feature207','feature208','feature209','feature210','feature211','feature212','feature213','feature214','feature215','feature216','feature217','feature218','feature219','feature220','feature221','feature222','feature223','feature224','feature225','feature226','feature227','feature228','feature229','feature230','feature231','feature232','feature233','feature234','feature235','feature236','feature237','feature238','feature239','feature240','feature241','feature242','feature243','feature244','feature245','feature246','feature247','feature248','feature249','feature250','feature251','feature252','feature253','feature254','feature255','feature256','feature257','feature258','feature259','feature260','feature261','feature262','feature263','feature264','feature265','feature266','feature267','feature268','feature269','feature270','feature271','feature272','feature273','feature274','feature275','feature276','feature277','feature278','feature279','feature280','feature281','feature282','feature283','feature284','feature285','feature286','feature287','feature288','feature289','feature290','feature291','feature292','feature293','feature294','feature295','feature296','feature297','feature298','feature299','feature300','feature301','feature302','feature303','feature304','feature305','feature306','feature307','feature308','feature309','feature310','feature311','feature312','feature313','feature314','feature315','feature316','feature317','feature318','feature319','feature320','feature321','feature322','feature323','feature324','feature325','feature326','feature327','feature328','feature329','feature330','feature331','feature332','feature333','feature334','feature335','feature336','feature337','feature338','feature339','feature340','feature341','feature342','feature343','feature344','feature345','feature346','feature347','feature348','feature349','feature350','feature351','feature352','feature353','feature354','feature355','feature356','feature357','feature358','feature359','feature360','feature361','feature362','feature363','feature364','feature365','feature366','feature367','feature368','feature369','feature370','feature371','feature372','feature373','feature374','feature375','feature376','feature377','feature378','feature379','feature380','feature381','feature382','feature383','feature384','feature385','feature386','feature387','feature388','feature389','feature390','feature391','feature392','feature393','feature394','feature395','feature396','feature397','feature398','feature399','feature400','feature401','feature402','feature403','feature404','feature405','feature406','feature407','feature408','feature409','feature410','feature411','feature412','feature413','feature414','feature415','feature416','feature417','feature418','feature419','feature420','feature421','feature422','feature423','feature424','feature425','feature426','feature427','feature428','feature429','feature430','feature431','feature432','feature433','feature434','feature435','feature436','feature437','feature438','feature439','feature440','feature441','feature442','feature443','feature444','feature445','feature446','feature447','feature448','feature449','feature450','feature451','feature452','feature453','feature454','feature455','feature456','feature457','feature458','feature459','feature460','feature461','feature462','feature463','feature464','feature465','feature466','feature467','feature468','feature469','feature470','feature471','feature472','feature473','feature474','feature475','feature476','feature477','feature478','feature479','feature480','feature481','feature482','feature483','feature484','feature485','feature486','feature487','feature488','feature489','feature490','feature491','feature492','feature493','feature494','feature495','feature496','feature497','feature498','feature499','feature500','feature501','feature502','feature503','feature504','feature505','feature506','feature507','feature508','feature509','feature510','feature511','feature512','feature513','feature514','feature515','feature516','feature517','feature518','feature519','feature520','feature521','feature522','feature523','feature524','feature525','feature526','feature527','feature528','feature529','feature530','feature531','feature532','feature533','feature534','feature535','feature536','feature537','feature538','feature539','feature540','feature541','feature542','feature543','feature544','feature545','feature546','feature547','feature548','feature549','feature550','feature551','feature552','feature553','feature554','feature555','feature556','feature557','feature558','feature559','feature560','feature561','feature562','feature563','feature564','feature565','feature566','feature567','feature568','feature569','feature570','feature571','feature572','feature573','feature574','feature575','feature576','feature577','feature578','feature579','feature580','feature581','feature582','feature583','feature584','feature585','feature586','feature587','feature588','feature589','feature590','feature591','feature592','feature593','feature594','feature595','feature596','feature597','feature598','feature599','feature600','feature601','feature602','feature603','feature604','feature605','feature606','feature607','feature608','feature609','feature610','feature611','feature612','feature613','feature614','feature615','feature616','feature617','feature618','feature619','feature620','feature621','feature622','feature623','feature624','feature625','feature626','feature627','feature628','feature629','feature630','feature631','feature632','feature633','feature634','feature635','feature636','feature637','feature638','feature639','feature640','feature641','feature642','feature643','feature644','feature645','feature646','feature647','feature648','feature649','feature650','feature651','feature652','feature653','feature654','feature655','feature656','feature657','feature658','feature659','feature660','feature661','feature662','feature663','feature664','feature665','feature666','feature667','feature668','feature669','feature670','feature671','feature672','feature673','feature674','feature675','feature676','feature677','feature678','feature679','feature680','feature681','feature682','feature683','feature684','feature685','feature686','feature687','feature688','feature689','feature690','feature691','feature692','feature693','feature694','feature695','feature696','feature697','feature698','feature699','feature700','feature701','feature702','feature703','feature704','feature705','feature706','feature707','feature708','feature709','feature710','feature711','feature712','feature713','feature714','feature715','feature716','feature717','feature718','feature719','feature720','feature721','feature722','feature723','feature724','feature725','feature726','feature727','feature728','feature729','feature730','feature731','feature732','feature733','feature734','feature735','feature736','feature737','feature738','feature739','feature740','feature741','feature742','feature743','feature744','feature745','feature746','feature747','feature748','feature749','feature750','feature751','feature752','feature753','feature754','feature755','feature756','feature757','feature758','feature759','feature760','feature761','feature762','feature763','feature764','feature765','feature766','feature767','feature768','feature769','feature770','feature771','feature772','feature773','feature774','feature775','feature776','feature777','feature778','feature779','feature780','feature781','feature782','feature783','feature784','feature785','feature786','feature787','feature788','feature789','feature790','feature791','feature792','feature793','feature794','feature795','feature796','feature797','feature798','feature799','feature800','feature801','feature802','feature803','feature804','feature805','feature806','feature807','feature808','feature809','feature810','feature811','feature812','feature813','feature814','feature815','feature816','feature817','feature818','feature819','feature820','feature821','feature822','feature823','feature824','feature825','feature826','feature827','feature828','feature829','feature830','feature831','feature832','feature833','feature834','feature835','feature836','feature837','feature838','feature839','feature840','feature841','feature842','feature843','feature844','feature845','feature846','feature847','feature848','feature849','feature850','feature851','feature852','feature853','feature854','feature855','feature856','feature857','feature858','feature859','feature860','feature861','feature862','feature863','feature864','feature865','feature866','feature867','feature868','feature869','feature870','feature871','feature872','feature873','feature874','feature875','feature876','feature877','feature878','feature879','feature880','feature881','feature882','feature883','feature884','feature885','feature886','feature887','feature888','feature889','feature890','feature891','feature892','feature893','feature894','feature895','feature896','feature897','feature898','feature899','feature900','feature901','feature902','feature903','feature904','feature905','feature906','feature907','feature908','feature909','feature910','feature911','feature912','feature913','feature914','feature915','feature916','feature917','feature918','feature919','feature920','feature921','feature922','feature923','feature924','feature925','feature926','feature927','feature928','feature929','feature930','feature931','feature932','feature933','feature934','feature935','feature936','feature937','feature938','feature939','feature940','feature941','feature942','feature943','feature944','feature945','feature946','feature947','feature948','feature949','feature950','feature951','feature952','feature953','feature954','feature955','feature956','feature957','feature958','feature959','feature960','feature961','feature962','feature963','feature964','feature965','feature966','feature967','feature968','feature969','feature970','feature971','feature972','feature973','feature974','feature975','feature976','feature977','feature978','feature979','feature980','feature981','feature982','feature983','feature984','feature985','feature986','feature987','feature988','feature989','feature990','feature991','feature992','feature993','feature994','feature995','feature996','feature997','feature998','feature999','feature1000','feature1001','feature1002','feature1003','feature1004','feature1005','feature1006','feature1007','feature1008','feature1009','feature1010','feature1011','feature1012','feature1013','feature1014','feature1015','feature1016','feature1017','feature1018','feature1019','feature1020','feature1021','feature1022','feature1023','feature1024','feature1025','feature1026','feature1027','feature1028','feature1029','feature1030','feature1031','feature1032','feature1033','feature1034','feature1035','feature1036','feature1037','feature1038','feature1039','feature1040','feature1041','feature1042','feature1043','feature1044','feature1045','feature1046','feature1047','feature1048','feature1049','feature1050','feature1051','feature1052','feature1053','feature1054','feature1055','feature1056','feature1057','feature1058','feature1059','feature1060','feature1061','feature1062','feature1063','feature1064','feature1065','feature1066','feature1067','feature1068','feature1069','feature1070','feature1071','feature1072','feature1073','feature1074','feature1075','feature1076','feature1077','feature1078','feature1079','feature1080','feature1081','feature1082','feature1083','feature1084','feature1085','feature1086','feature1087','feature1088','feature1089','feature1090','feature1091','feature1092','feature1093','feature1094','feature1095','feature1096','feature1097','feature1098','feature1099','feature1100','feature1101','feature1102','feature1103','feature1104','feature1105','feature1106','feature1107','feature1108','feature1109','feature1110','feature1111','feature1112','feature1113','feature1114','feature1115','feature1116','feature1117','feature1118','feature1119','feature1120','feature1121','feature1122','feature1123','feature1124','feature1125','feature1126','feature1127','feature1128','feature1129','feature1130','feature1131','feature1132','feature1133','feature1134','feature1135','feature1136','feature1137','feature1138','feature1139','feature1140','feature1141','feature1142','feature1143','feature1144','feature1145','feature1146','feature1147','feature1148','feature1149','feature1150','feature1151','feature1152','feature1153','feature1154','feature1155','feature1156','feature1157','feature1158','feature1159','feature1160','feature1161','feature1162','feature1163','feature1164','feature1165','feature1166','feature1167','feature1168','feature1169','feature1170','feature1171','feature1172','feature1173','feature1174','feature1175','feature1176','feature1177','feature1178','feature1179','feature1180','feature1181','feature1182','feature1183','feature1184','feature1185','feature1186','feature1187','feature1188','feature1189','feature1190','feature1191','feature1192','feature1193','feature1194','feature1195','feature1196','feature1197','feature1198','feature1199','feature1200','feature1201','feature1202','feature1203','feature1204','feature1205','feature1206','feature1207','feature1208','feature1209','feature1210','feature1211','feature1212','feature1213','feature1214','feature1215','feature1216','feature1217','feature1218','feature1219','feature1220','feature1221','feature1222','feature1223','feature1224','feature1225','feature1226','feature1227','feature1228','feature1229','feature1230','feature1231','feature1232','feature1233','feature1234','feature1235','feature1236','feature1237','feature1238','feature1239','feature1240','feature1241','feature1242','feature1243','feature1244','feature1245','feature1246','feature1247','feature1248','feature1249','feature1250','feature1251','feature1252','feature1253','feature1254','feature1255','feature1256','feature1257','feature1258','feature1259','feature1260','feature1261','feature1262','feature1263','feature1264','feature1265','feature1266','feature1267','feature1268','feature1269','feature1270','feature1271','feature1272','feature1273','feature1274','feature1275','feature1276','feature1277','feature1278','feature1279','feature1280','feature1281','feature1282','feature1283','feature1284','feature1285','feature1286','feature1287','feature1288','feature1289','feature1290','feature1291','feature1292','feature1293','feature1294','feature1295','feature1296','feature1297','feature1298','feature1299','feature1300','feature1301','feature1302','feature1303','feature1304','feature1305','feature1306','feature1307','feature1308','feature1309','feature1310','feature1311','feature1312','feature1313','feature1314','feature1315','feature1316','feature1317','feature1318','feature1319','feature1320','feature1321','feature1322','feature1323','feature1324','feature1325','feature1326','feature1327','feature1328','feature1329','feature1330','feature1331','feature1332','feature1333','feature1334','feature1335','feature1336','feature1337','feature1338','feature1339','feature1340','feature1341','feature1342','feature1343','feature1344','feature1345','feature1346','feature1347','feature1348','feature1349','feature1350','feature1351','feature1352','feature1353','feature1354','feature1355','feature1356','feature1357','feature1358','feature1359','feature1360','feature1361','feature1362','feature1363','feature1364','feature1365','feature1366','feature1367','feature1368','feature1369','feature1370','feature1371','feature1372','feature1373','feature1374','feature1375','feature1376','feature1377','feature1378','feature1379','feature1380','feature1381','feature1382','feature1383','feature1384','feature1385','feature1386','feature1387','feature1388','feature1389','feature1390','feature1391','feature1392','feature1393','feature1394','feature1395','feature1396','feature1397','feature1398','feature1399','feature1400','feature1401','feature1402','feature1403','feature1404','feature1405','feature1406','feature1407','feature1408','feature1409','feature1410','feature1411','feature1412','feature1413','feature1414','feature1415','feature1416','feature1417','feature1418','feature1419','feature1420','feature1421','feature1422','feature1423','feature1424','feature1425','feature1426','feature1427','feature1428','feature1429','feature1430','feature1431','feature1432','feature1433','feature1434','feature1435','feature1436','feature1437','feature1438','feature1439','feature1440','feature1441','feature1442','feature1443','feature1444','feature1445','feature1446','feature1447','feature1448','feature1449','feature1450','feature1451','feature1452','feature1453','feature1454','feature1455','feature1456','feature1457','feature1458','feature1459','feature1460','feature1461','feature1462','feature1463','feature1464','feature1465','feature1466','feature1467','feature1468','feature1469','feature1470','feature1471','feature1472','feature1473','feature1474','feature1475','feature1476','feature1477','feature1478','feature1479','feature1480','feature1481','feature1482','feature1483','feature1484','feature1485','feature1486','feature1487','feature1488','feature1489','feature1490','feature1491','feature1492','feature1493','feature1494','feature1495','feature1496','feature1497','feature1498','feature1499','feature1500','feature1501','feature1502','feature1503','feature1504','feature1505','feature1506','feature1507','feature1508','feature1509','feature1510','feature1511','feature1512','feature1513','feature1514','feature1515','feature1516','feature1517','feature1518','feature1519','feature1520','feature1521','feature1522','feature1523','feature1524','feature1525','feature1526','feature1527','feature1528','feature1529','feature1530','feature1531','feature1532','feature1533','feature1534','feature1535','feature1536','feature1537','feature1538','feature1539','feature1540','feature1541','feature1542','feature1543','feature1544','feature1545','feature1546','feature1547','feature1548','feature1549','feature1550','feature1551','feature1552','feature1553','feature1554','feature1555','feature1556','feature1557','feature1558','feature1559','feature1560','feature1561','feature1562','feature1563','feature1564','feature1565','feature1566','feature1567','feature1568','feature1569','feature1570','feature1571','feature1572','feature1573','feature1574','feature1575','feature1576','feature1577','feature1578','feature1579','feature1580','feature1581','feature1582','feature1583','feature1584','feature1585','feature1586','feature1587','feature1588','feature1589','feature1590','feature1591','feature1592','feature1593','feature1594','feature1595','feature1596','feature1597','feature1598','feature1599','feature1600','feature1601','feature1602','feature1603','feature1604','feature1605','feature1606','feature1607','feature1608','feature1609','feature1610','feature1611','feature1612','feature1613','feature1614','feature1615','feature1616','feature1617','feature1618','feature1619','feature1620','feature1621','feature1622','feature1623','feature1624','feature1625','feature1626','feature1627','feature1628','feature1629','feature1630','feature1631','feature1632','feature1633','feature1634','feature1635','feature1636','feature1637','feature1638','feature1639','feature1640','feature1641','feature1642','feature1643','feature1644','feature1645','feature1646','feature1647','feature1648','feature1649','feature1650','feature1651','feature1652','feature1653','feature1654','feature1655','feature1656','feature1657','feature1658','feature1659','feature1660','feature1661','feature1662','feature1663','feature1664','feature1665','feature1666','feature1667','feature1668','feature1669','feature1670','feature1671','feature1672','feature1673','feature1674','feature1675','feature1676','feature1677','feature1678','feature1679','feature1680','feature1681','feature1682','feature1683','feature1684','feature1685','feature1686','feature1687','feature1688','feature1689','feature1690','feature1691','feature1692','feature1693','feature1694','feature1695','feature1696','feature1697','feature1698','feature1699','feature1700','feature1701','feature1702','feature1703','feature1704','feature1705','feature1706','feature1707','feature1708','feature1709','feature1710','feature1711','feature1712','feature1713','feature1714','feature1715','feature1716','feature1717','feature1718','feature1719','feature1720','feature1721','feature1722','feature1723','feature1724','feature1725','feature1726','feature1727','feature1728','feature1729','feature1730','feature1731','feature1732','feature1733','feature1734','feature1735','feature1736','feature1737','feature1738','feature1739','feature1740','feature1741','feature1742','feature1743','feature1744','feature1745','feature1746','feature1747','feature1748','feature1749','feature1750','feature1751','feature1752','feature1753','feature1754','feature1755','feature1756','feature1757','feature1758','feature1759','feature1760','feature1761','feature1762','feature1763','feature1764','feature1765','feature1766','feature1767','feature1768','feature1769','feature1770','feature1771','feature1772','feature1773','feature1774','feature1775','feature1776','feature1777','feature1778','feature1779','feature1780','feature1781','feature1782','feature1783','feature1784','feature1785','feature1786','feature1787','feature1788','feature1789','feature1790','feature1791','feature1792','feature1793','feature1794','feature1795','feature1796','feature1797','feature1798','feature1799','feature1800','feature1801','feature1802','feature1803','feature1804','feature1805','feature1806','feature1807','feature1808','feature1809','feature1810','feature1811','feature1812','feature1813','feature1814','feature1815','feature1816','feature1817','feature1818','feature1819','feature1820','feature1821','feature1822','feature1823','feature1824','feature1825','feature1826','feature1827','feature1828','feature1829','feature1830','feature1831','feature1832','feature1833','feature1834','feature1835','feature1836','feature1837','feature1838','feature1839','feature1840','feature1841','feature1842','feature1843','feature1844','feature1845','feature1846','feature1847','feature1848','feature1849','feature1850','feature1851','feature1852','feature1853','feature1854','feature1855','feature1856','feature1857','feature1858','feature1859','feature1860','feature1861','feature1862','feature1863','feature1864','feature1865','feature1866','feature1867','feature1868','feature1869','feature1870','feature1871','feature1872','feature1873','feature1874','feature1875','feature1876','feature1877','feature1878','feature1879','feature1880','feature1881','feature1882','feature1883','feature1884','feature1885','feature1886','feature1887','feature1888','feature1889','feature1890','feature1891','feature1892','feature1893','feature1894','feature1895','feature1896','feature1897','feature1898','feature1899','feature1900','feature1901','feature1902','feature1903','feature1904','feature1905','feature1906','feature1907','feature1908','feature1909','feature1910','feature1911','feature1912','feature1913','feature1914','feature1915','feature1916','feature1917','feature1918','feature1919','feature1920','feature1921','feature1922','feature1923','feature1924','feature1925','feature1926','feature1927','feature1928','feature1929','feature1930','feature1931','feature1932','feature1933','feature1934','feature1935','feature1936','feature1937','feature1938','feature1939','feature1940','feature1941','feature1942','feature1943','feature1944','feature1945','feature1946','feature1947','feature1948','feature1949','feature1950','feature1951','feature1952','feature1953','feature1954','feature1955','feature1956','feature1957','feature1958','feature1959','feature1960','feature1961','feature1962','feature1963','feature1964','feature1965','feature1966','feature1967','feature1968','feature1969','feature1970','feature1971','feature1972','feature1973','feature1974','feature1975','feature1976','feature1977','feature1978','feature1979','feature1980','feature1981','feature1982','feature1983','feature1984','feature1985','feature1986','feature1987','feature1988','feature1989','feature1990','feature1991','feature1992','feature1993','feature1994','feature1995','feature1996','feature1997','feature1998','feature1999','feature2000','feature2001','feature2002','feature2003','feature2004','feature2005','feature2006','feature2007','feature2008','feature2009','feature2010','feature2011','feature2012','feature2013','feature2014','feature2015','feature2016','feature2017','feature2018','feature2019','feature2020','feature2021','feature2022','feature2023','feature2024','feature2025','feature2026','feature2027','feature2028','feature2029','feature2030','feature2031','feature2032','feature2033','feature2034','feature2035','feature2036','feature2037','feature2038','feature2039','feature2040','feature2041','feature2042','feature2043','feature2044','feature2045','feature2046','feature2047','feature2048']

In [10]:
#Resnet training phase
model = ResNet50(weights='imagenet',include_top = False, pooling = "max")

In [6]:
ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
data = list(csv.reader(ClassInfo))
NumClass = int(data[0][0]) #No of class
count = 0 #Used in writing frames to text file

In [16]:
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTrainDataSet = int(data[k+1][1])  #No of Training datasets
   
    #Trainingdata   
    for i in range(NumTrainDataSet): #Looping through all datasets in a class  
        FixationInfo = pd.read_csv(FilePath + "Dataset/Dataset_Preprocessed/Fixations/" + "C"+str(k)+"_TR_"+str(i)+"_fixations.csv", usecols=[0])
        data = FixationInfo.values
        fixation_data = [item for sublist in data for item in sublist]
        NumFixation = len(fixation_data)-1 #no of fixations in a data set
        
        #Computing Feature Vector for Target Data
        TargetImagePath = FilePath + "Runs/" + RunFolder + "/Frames_Resized/" + "C" + str(k) +"_TR_" + str(i) + "_T_" + str(i) + ext
        TargetImage = imge.load_img(TargetImagePath, target_size=(width, height))
        x = imge.img_to_array(TargetImage)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        TempTargetFeatureVector = model.predict(x)                               
        TargetFeatureVector = [item for sublist in TempTargetFeatureVector for item in sublist] #To convert nd array into list
        NPTargetFeatureVector = np.array([TargetFeatureVector])

        #Writing Feature Vector of each data set in a separate file, no of rows = no of data set  
        with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Target_Combined_Csv/TR_CombinedTarget_FeatureVector.csv", 'a', newline='') as graphFile:     
            graphFileWriter = csv.writer(graphFile, delimiter = ',')
            graphFileWriter.writerow(TargetFeatureVector)
        
        #Generating target feature vectors into panda frames
        row_frame = pd.DataFrame({col_names[0]:(str(k))},index=[0])
        for s in range(1,len(TargetFeatureVector)+1):
                 tmp = pd.DataFrame({col_names[s]:NPTargetFeatureVector[:,s-1]})
                 row_frame = row_frame.join(tmp)
        if count == 0:
            table_frame = row_frame
            count=1
        else:
            table_frame = table_frame.append(row_frame)  
            
        print("---------------------------C" + str(k) +"_TR_" + str(i) + "_T_" + str(i))
                
        #Fixation duration total
        FixationDuration = pd.read_csv(FilePath + "Dataset/Dataset_Preprocessed/Fixations/" + "C"+str(k)+"_TR_"+ str(i) +"_fixations.csv", usecols=[2])
        data = FixationDuration.values
        NormFixationDurationList = [item for sublist in data for item in sublist]

        TotalDuration = sum(NormFixationDurationList) - NormFixationDurationList[NumFixation]
        NormFixationDurationList[:] = [x / TotalDuration for x in NormFixationDurationList]   
        
        #Computing Feature Vector for Fixation Data       
        for j in range(NumFixation):#Lopping through all fixations
            FixationImagePath = FilePath + "Runs/" + RunFolder + "/Frames_Resized/" + "C" + str(k)+"_TR_" + str(i) + "_F_" + str(j) + ext
            FixationImage = imge.load_img(FixationImagePath, target_size=(width, height))
            x = imge.img_to_array(FixationImage)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            print("C" + str(k)+"_TR_" + str(i) + "_F_" + str(j) + ext)
            
            TempFeatureVector = model.predict(x)                               
            FeatureVector = [item for sublist in TempFeatureVector for item in sublist] #To convert nd array into list
            
            #Writing Feature Vector of each data set in a separate file, no of rows = no of fixations
            with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_Original/C" + str(k) + "_TR_" + str(i) + '_Fixation_FeatureVector.csv', 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(FeatureVector)    
                
            #multiply FeatureVector with normalized fixation duration weight
            FeatureVector = [x * NormFixationDurationList[j] for x in FeatureVector]
            #Writing Feature Vector of each data set in a separate file, no of rows = no of fixations    
            with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_WeightNormalized/C" + str(k) + "_TR_" + str(i) + '_Fixation_FeatureVector.csv', 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(FeatureVector) 
                
        #Computing Combined Feature Vector
        FixationFeatureFile = open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_WeightNormalized/C" + str(k)+"_TR_" + str(i) + '_Fixation_FeatureVector.csv',"r")
        FeaturePoint = list(csv.reader(FixationFeatureFile))
        CombinedFeatureVector = [0]*len(FeaturePoint[0])
        for m in range(len(FeaturePoint[0])):
            for n in range(NumFixation):
                CombinedFeatureVector[m] += float(FeaturePoint[n][m])
                
        #Writing combined Feature Vector of each data set in a file, no of rows = no of data sets          
        with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_WeightedSum_Combined_Csv/TR_CombinedFixation_FeatureVector.csv", 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(CombinedFeatureVector) 


#Writing Appended Target Feature Vector Frames into a text file
table_frame.to_csv(FilePath  + "Runs/" + RunFolder + "/FV2048_Resnet/Target_TR_Classifier.txt", sep='\t', index=False)

C0_TR_0_F_0.jpg
C0_TR_0_F_1.jpg
C0_TR_0_F_2.jpg
C3_TR_0_F_0.jpg
C3_TR_0_F_1.jpg
C3_TR_0_F_2.jpg
C3_TR_0_F_3.jpg
C3_TR_0_F_4.jpg
C3_TR_0_F_5.jpg
C3_TR_0_F_6.jpg
C4_TR_0_F_0.jpg
C4_TR_0_F_1.jpg
C4_TR_0_F_2.jpg
C4_TR_0_F_3.jpg
C4_TR_0_F_4.jpg
C4_TR_0_F_5.jpg
C4_TR_0_F_6.jpg
C4_TR_0_F_7.jpg
C4_TR_0_F_8.jpg
C4_TR_0_F_9.jpg
C4_TR_0_F_10.jpg
C4_TR_0_F_11.jpg
C4_TR_0_F_12.jpg
C4_TR_0_F_13.jpg
C4_TR_0_F_14.jpg
C4_TR_0_F_15.jpg
C5_TR_0_F_0.jpg
C5_TR_0_F_1.jpg
C5_TR_0_F_2.jpg
C6_TR_0_F_0.jpg
C6_TR_0_F_1.jpg
C6_TR_0_F_2.jpg
C6_TR_0_F_3.jpg
C6_TR_0_F_4.jpg
C6_TR_0_F_5.jpg
C7_TR_0_F_0.jpg
C7_TR_0_F_1.jpg
C7_TR_0_F_2.jpg
C7_TR_0_F_3.jpg


TypeError: bad operand type for unary +: 'str'

In [ ]:
#Resnet testing phase
ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
data = list(csv.reader(ClassInfo))
NumClass = int(data[0][0]) #No of class
count = 0

In [21]:

for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTestDataSet = int(data[k+1][2])  #No of Training datasets
   
    #Testing data   
    for i in range(NumTestDataSet): #Looping through all datasets in a class  
        FixationInfo = pd.read_csv(FilePath + "Dataset/Dataset_Preprocessed/Fixations/"  +"C"+str(k)+"_Test_"+str(i)+"_fixations.csv", usecols=[0])
        data = FixationInfo.values
        fixation_data = [item for sublist in data for item in sublist]
        NumFixation = len(fixation_data)-1 #no of fixations in a data set
        
        #Computing Feature Vector for Target Data
        TargetImagePath = FilePath + "Runs/" + RunFolder + "/Frames_Resized/"  + "C" + str(k) +"_Test_" + str(i) + "_T_" + str(i) + ext
        TargetImage = imge.load_img(TargetImagePath, target_size=(width, height))
        x = imge.img_to_array(TargetImage)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        TempTargetFeatureVector = model.predict(x)                               
        TargetFeatureVector = [item for sublist in TempTargetFeatureVector for item in sublist] #To convert nd array into list
        
        #Writing Feature Vector of each data set in a separate file, no of rows = no of data set  
        with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Target_Combined_Csv/Test_CombinedTarget_FeatureVector.csv", 'a', newline='') as graphFile:     
            graphFileWriter = csv.writer(graphFile, delimiter = ',')
            graphFileWriter.writerow(TargetFeatureVector)            
        
        print("---------------------------C" + str(k) +"_Test_" + str(i) + "_T_" + str(i))
                  
        #Fixation duration total
        FixationDuration = pd.read_csv(FilePath + "Dataset/Dataset_Preprocessed/Fixations/" + "C"+str(k)+"_Test_"+ str(i) +"_fixations.csv", usecols=[2])
        data = FixationDuration.values
        NormFixationDurationList = [item for sublist in data for item in sublist]

        TotalDuration = sum(NormFixationDurationList)- NormFixationDurationList[NumFixation]
        NormFixationDurationList[:] = [x / TotalDuration for x in NormFixationDurationList]   
        
        #Computing Feature Vector for Fixation Data       
        for j in range(NumFixation):#Lopping through all fixations
            FixationImagePath = FilePath + "Runs/" + RunFolder + "/Frames_Resized/" + "C" + str(k)+"_Test_" + str(i) + "_F_" + str(j) + ext
            FixationImage = imge.load_img(FixationImagePath, target_size=(width, height))
            x = imge.img_to_array(FixationImage)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            TempFeatureVector = model.predict(x)                               
            FeatureVector = [item for sublist in TempFeatureVector for item in sublist] #To convert nd array into list
            
            #Writing Feature Vector of each data set in a separate file, no of rows = no of fixations
            with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_Original/C" + str(k) + "_Test_" + str(i) + '_Fixation_FeatureVector.csv', 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(FeatureVector) 
            
              #multiply FeatureVector with normalized fixation duration weight
            FeatureVector = [x * NormFixationDurationList[j] for x in FeatureVector]
            #Writing Feature Vector of each data set in a separate file, no of rows = no of fixations    
            with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_WeightNormalized/C" + str(k) + "_Test_" + str(i) + '_Fixation_FeatureVector.csv', 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(FeatureVector) 
                
        #Computing Combined Feature Vector
        FixationFeatureFile = open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_WeightNormalized/C" + str(k)+"_Test_" + str(i) + '_Fixation_FeatureVector.csv',"r")
        FeaturePoint = list(csv.reader(FixationFeatureFile))
        CombinedFeatureVector = [0]*len(FeaturePoint[0])
        for m in range(len(FeaturePoint[0])):
            for n in range(NumFixation):
                CombinedFeatureVector[m] += float(FeaturePoint[n][m])
                
        #Writing combined Feature Vector of each data set in a file, no of rows = no of data sets          
        with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_WeightedSum_Combined_Csv/Test_CombinedFixation_FeatureVector.csv", 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(CombinedFeatureVector) 
                


---------------------------C3_Test_0_T_0


In [38]:
#creat a label file csv

#Training
TotalNumTRSet = 0
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTRDataSet = int(data[k+1][1])  #No of Training datasets
    
    TotalNumTRSet = TotalNumTRSet + NumTRDataSet
    
TRLabel = [0]*TotalNumTRSet

TRCount = 0
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTRDataSet = int(data[k+1][1])  #No of Training datasets
    
    for i in range(NumTRDataSet): #Looping through all datasets in a class  
        TRLabel[TRCount] = k
        TRCount = TRCount +1

#Write in  a file
        with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/TR_Label.csv", 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(str(k))        

#Testing
TotalNumTestSet = 0
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTestDataSet = int(data[k+1][2])  #No of Testing datasets
    
    TotalNumTestSet = TotalNumTestSet + NumTestDataSet
    
TestLabel = [0]*TotalNumTestSet

TestCount = 0
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTestDataSet = int(data[k+1][2])  #No of Training datasets
    
    for i in range(NumTestDataSet): #Looping through all datasets in a class  
        TestLabel[TestCount] = k
        TestCount = TestCount +1
        
        #Write in  a file
        with open(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Test_Label.csv", 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(str(k))     